### Cheking if we are in google colab or not.

In [1]:
import sys
IN_COLAB = 'google.colab' in sys.modules
IN_COLAB

False

In [2]:
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    !pip install geemap
    !pip install geedim

In [3]:
import ee
import geemap

In [4]:
from datetime import date, datetime
from dateutil.relativedelta import relativedelta
from skimage import io
import numpy as np
import ee
import geemap
import logging
import multiprocessing
import os
import requests
import shutil
# from retry import retry
from datetime import datetime
import matplotlib.pyplot as plt

from collections import namedtuple

In [5]:
from utils.utils import *
from utils.ee_utils import *
from utils.ee_utils import TextColors as tc

4/1AfgeXvvmWF-_z0qOvwsZK597gUnHMOdSTjpY6CStI-buWxqNvRlGoRU19og

In [6]:
Map = geemap.Map()

In [7]:
#ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')
ee.Initialize()

# A Function to retrive S1 and S2

has two inputs:
*   date
*   roi


Returns:
*   s1
*   s2 image




In [8]:
from datetime import date, datetime
from dateutil.relativedelta import relativedelta

---

# OSCD Dataset ROIs

## aguasclaras

In [9]:
from rois import ROIS
rois = ROIS()
roi = rois.abudhabi

In [10]:
features = geemap.fishnet(roi, rows=2, cols=2, delta=0.1)
#features = geemap.fishnet(roi, h_interval=0.05, v_interval=0.05)

In [11]:
Map = geemap.Map(zoom=12)
Map.add_basemap('SATELLITE')
Map.addLayer(roi)
Map.addLayer(rois.abudhabi,{'color': '45ff45'})
Map.addLayer(rois.abudhabi_buffed,{'color': '22ff22'})
Map.addLayer(rois.aguasclaras,{'color': 'fc03ff'})
Map.addLayer(rois.aguasclaras_adj_goiania,{'color': 'ff03ff'})
Map.centerObject(roi)
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [12]:
# 'ASCENDING'
# 'DESCENDING'
s2_col,s1_col = s1s2(roi, date = ('2021-04-07','2021-07-07'),priority_path = 'ASCENDING',max_cloud = 100,max_snow = 100,month_span = 2,retry_days=0)
# s2_col,s1_col_desc = s1s2(roi,date = ('2020-08-10', '2020-09-20'),priority_path = 'DESCENDING',max_cloud = 5)

 ◍◍Finding S2 
 ◍Single scene coverege was found! 
 ◍◍Finding S1 
mean s2 date:  2021-05-22
final date range:   2021-03-22 2021-07-22 | diff_days -> 122 
◍checking for ASCENDING single scene
Mosiac Covers 96.80772245612427 percent of the roi.
Mosaic Coverege Accepted
Mosaic Coverege Accepted
 ◍Collection Found! 


In [22]:
s1_col = ee.ImageCollection("COPERNICUS/S1_GRD") .filterDate('2021-04-07','2021-07-07').filterBounds(roi)
s1_col.size().getInfo()

15

In [26]:
# Get the download URL for the image
url = s1_img_vv.getDownloadURL({
    'scale': 10,
    'crs': 'EPSG:4326',
    'region': roi
})

# Download the image as a TIFF file without geographic information
response = requests.get(url)
with open('s11image.tif', 'wb') as f:
    f.write(response.content)

In [27]:
s1_img.getInfo()

{'type': 'Image',
 'bands': [{'id': 'VV',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'VH',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'angle',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}]}

In [24]:
s1_img = s1_col.mean()
s1_img_vv = s1_img.select('VV').clip(roi)
geemap.download_ee_image(s1_img_vv, "s11image.tif",crs='EPSG:4326', scale=10,region = roi)

TypeError: _prepare_for_export() got an unexpected keyword argument 'geotiff'

In [18]:
bands = ['B2','B3','B4']
s2_img = s2_col.first()
s2_img = s2_img.select(bands)
geemap.download_ee_image(s2_img, "s2image.tif",crs='EPSG:4326', scale=10,region = roi)

s2image.tif: |          | 0.00/4.13M (raw) [  0.0%] in 00:00 (eta:     ?)

In [17]:
s1_col.mean().propertyNames().getInfo()

['system:bands', 'system:band_names']

In [13]:
sen2_print(s2_col)

,Property,0,1
0,system:time_start,2021-04-08,2021-07-02
1,roi_cloud_cover,1.026529,1.244256
2,CLOUDY_PIXEL_PERCENTAGE,1.043424,1.087634
3,CLOUD_SHADOW_PERCENTAGE,0.035080,0.000342
4,VEGETATION_PERCENTAGE,0.470002,0.123719
5,NOT_VEGETATED_PERCENTAGE,91.787839,91.874266
6,CLOUD_COVERAGE_ASSESSMENT,1.043424,1.087634
7,GENERATION_TIME,1617873490000,1625218895000
8,SENSING_ORBIT_NUMBER,20,20
9,NODATA_PIXEL_PERCENTAGE,0.000007,0


In [14]:
sen1_print(s1_col.sort('system:time_start')) #.filter(ee.Filter.eq('relativeOrbitNumber_start', 30))

,Property,0,1,2,3,4,5,6,7
0,system:time_start,2021-04-04,2021-04-16,2021-04-28,2021-05-10,2021-05-22,2021-06-03,2021-06-15,2021-06-27
1,orbitProperties_pass,ASCENDING,ASCENDING,ASCENDING,ASCENDING,ASCENDING,ASCENDING,ASCENDING,ASCENDING
2,resolution,H,H,H,H,H,H,H,H
3,resolution_meters,10,10,10,10,10,10,10,10
4,platform_number,A,A,A,A,A,A,A,A
5,productType,GRD,GRD,GRD,GRD,GRD,GRD,GRD,GRD
6,orbitNumber_start,37302,37477,37652,37827,38002,38177,38352,38527
7,orbitNumber_stop,37302,37477,37652,37827,38002,38177,38352,38527
8,transmitterReceiverPolarisation,"['VV', 'VH']","['VV', 'VH']","['VV', 'VH']","['VV', 'VH']","['VV', 'VH']","['VV', 'VH']","['VV', 'VH']","['VV', 'VH']"
9,system:band_names,"['VV', 'VH', 'angle']","['VV', 'VH', 'angle']","['VV', 'VH', 'angle']","['VV', 'VH', 'angle']","['VV', 'VH', 'angle']","['VV', 'VH', 'angle']","['VV', 'VH', 'angle']","['VV', 'VH', 'angle']"


In [15]:
vis_params_s2 = {
  'min': 0.04,
  'max': 0.4,
  'bands': ['B4', 'B3', 'B2'],
}

# un comment this for Band 8 to be added to the map so you can check the correlation between NIR and radar backscatter

# vis_params_s2 = {
#   'min': 0.04,
#   'max': 0.4,
#   'bands': ['B8'],
# }

vis_params_s1_dark = {
    'min': -18,
    'max': -2,
    'bands': ['VV_db']
}

# vis params for dense urban structure, comment if images is too dark. (so the above one gets applied)
vis_params_s1_bright = {
    'min': -10,
    'max': 10,
    'bands': ['VV_db']
}

## To Linear and back again!
as was said in the `Woodhouse pg. 324` we cannot averasge over `db` values.
so we convert the data into `linear` scale, then we use `.mean()` method

In [16]:
s1_col = s1_col.map(toLinear)

In [17]:
sen1_print(s1_col)

,Property,0,1,2,3,4,5,6,7
0,system:time_start,2021-04-04,2021-04-16,2021-04-28,2021-05-10,2021-05-22,2021-06-03,2021-06-15,2021-06-27
1,orbitProperties_pass,ASCENDING,ASCENDING,ASCENDING,ASCENDING,ASCENDING,ASCENDING,ASCENDING,ASCENDING
2,resolution,H,H,H,H,H,H,H,H
3,resolution_meters,10,10,10,10,10,10,10,10
4,platform_number,A,A,A,A,A,A,A,A
5,productType,GRD,GRD,GRD,GRD,GRD,GRD,GRD,GRD
6,orbitNumber_start,37302,37477,37652,37827,38002,38177,38352,38527
7,orbitNumber_stop,37302,37477,37652,37827,38002,38177,38352,38527
8,transmitterReceiverPolarisation,"['VV', 'VH']","['VV', 'VH']","['VV', 'VH']","['VV', 'VH']","['VV', 'VH']","['VV', 'VH']","['VV', 'VH']","['VV', 'VH']"
9,system:band_names,"['VV', 'VH', 'angle', 'VV_lin']","['VV', 'VH', 'angle', 'VV_lin']","['VV', 'VH', 'angle', 'VV_lin']","['VV', 'VH', 'angle', 'VV_lin']","['VV', 'VH', 'angle', 'VV_lin']","['VV', 'VH', 'angle', 'VV_lin']","['VV', 'VH', 'angle', 'VV_lin']","['VV', 'VH', 'angle', 'VV_lin']"


In [18]:
s1_img = s1_col.mean().clip(roi)
s1_img = toDb(s1_img)
s2_img = s2_col.median().clip(roi).multiply(0.0001)

In [21]:
left_layer = geemap.ee_tile_layer(s1_img, vis_params_s1_bright, 'S1')
right_layer = geemap.ee_tile_layer(s2_img, vis_params_s2, 'S2')

### Folliom
when I use center object roi it takes some time to zoom in, the work around is to used `zoom` argument in the `Map = geemap.Map(zoom=12)` this way, the map is already zoomed in and this way you can buy yourself some time.

In [22]:
Map = geemap.Map(zoom=12)
Map.centerObject(roi)
Map.split_map(left_layer, right_layer)
Map

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

### Sentinel 2 Surface Reflectance band 10
band 10 is cirus and is available in TOA data, and is used for atmospheric correction, therfore it is not in the SR img.

In [20]:
bands = ['B1','B2','B3','B4','B5','B6','B7','B8','B8A','B9','B11','B12']

In [21]:
s1_img_vv = s1_img.select('VV_db')
s2_img_13b = s2_img.select(bands)

In [22]:
print(s1_img_vv.bandNames().getInfo())
print(s2_img_13b.bandNames().getInfo())

['VV_db']
['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12']


In [23]:
#geemap.download_ee_image(s1_img_vv, "s1.tif",crs='EPSG:4326', scale=10,region = roi)

In [24]:
#geemap.download_ee_image(s2_img_13b, "s2.tif",crs='EPSG:4326', scale=10,region = roi)

In [25]:
# s1_np = io.imread('s1.tif')
# s2_np = io.imread('s2.tif')

# print(s1_np.shape)
# print(s2_np.shape)

In [26]:
# print(np.min(s1_np),np.mean(s1_np),np.max(s1_np))
# print(np.min(s2_np),np.max(s2_np))

In [27]:
# plt.imshow(s1_np)
# plt.colorbar()

In [28]:
# plt.hist(s1_np)

In [29]:
# s2_np_swaped = np.swapaxes(s2_np,2,0)
# s2_np_swaped = np.swapaxes(s2_np_swaped,1,0)
# print(s2_np_swaped.shape)

In [30]:
# s2_np_oneband = s2_np_swaped[:,:,1]
# print(np.min(s2_np_oneband),np.max(s2_np_oneband))

In [31]:
# print(s2_np_oneband.shape)
# plt.imshow(s2_np_oneband)
# plt.colorbar()

# Downlaod All!

In [32]:
import pandas as pd
import ast
import collections
import os
import shutil

In [63]:
YEAR = 2021
Y = YEAR - 2000
rois_path = f'D:\python\TemporalGAN\dataset\\{Y}_ROIs.xlsx'
SR = False if YEAR == 2017 else True

In [64]:
df = pd.read_excel(rois_path)

## checking if there is a duplicate in the names

In [65]:
names = []
for index, row in df.iterrows():
    names.append(row['name']+str(row['year_type']))
print([item for item, count in collections.Counter(names).items() if count > 1])

[]


In [66]:
#shutil.rmtree('/content/s2_imgs/')  

In [67]:
# Consice info printing function
sen2_short_print = lambda s2_collection: ee_property_printer(s2_collection, propertie_name_list=['system:time_start','roi_cloud_cover', 'CLOUDY_PIXEL_PERCENTAGE',
                                                                                                            'system:band_names'],
                                                             first_is_t_in_millis=True, df_instead_of_print=False)

sen1_short_print = lambda s2_collection: ee_property_printer(s2_collection, propertie_name_list=['system:time_start','platform_number',
                                                                                           'system:band_names','instrumentMode',                         
                                                                                            'relativeOrbitNumber_stop','relativeOrbitNumber_start',
                                                                                            'cycleNumber'],
                                                             first_is_t_in_millis=True, df_instead_of_print=False)

### dataset downlaoder

In [68]:
import os

current_dir = os.getcwd()

print(current_dir)

d:\python\TemporalGAN\dataset


In [69]:
if IN_COLAB:
    S1_FOLDER_TRAIN = f'/content/{YEAR}/s1_imgs/train/'
    S1_FOLDER_TEST = f'/content/{YEAR}/s1_imgs/test/'
    S2_FOLDER_TRAIN = f'/content/{YEAR}/s2_imgs/train/'
    S2_FOLDER_TEST = f'/content/{YEAR}/s2_imgs/test/'
else:
    S1_FOLDER_TRAIN = f'./s1s2/{YEAR}/s1_imgs/train/'
    S1_FOLDER_TEST = f'./s1s2/{YEAR}/s1_imgs/test/'
    S2_FOLDER_TRAIN = f'./s1s2/{YEAR}/s2_imgs/train/'
    S2_FOLDER_TEST = f'./s1s2/{YEAR}/s2_imgs/test/'
# create folders if not exist
if not os.path.exists(S1_FOLDER_TRAIN) or not os.path.exists(S1_FOLDER_TEST) or not os.path.exists(S2_FOLDER_TRAIN) or not os.path.exists(S2_FOLDER_TEST):
    print("Creating folders...")
    os.makedirs(S1_FOLDER_TRAIN, exist_ok=True)
    os.makedirs(S1_FOLDER_TEST, exist_ok=True)
    os.makedirs(S2_FOLDER_TRAIN, exist_ok=True)
    os.makedirs(S2_FOLDER_TEST, exist_ok=True)

In [70]:
#df = df.iloc[5:]

In [71]:
bands = ['B2','B3','B4','B8','B11','B12']
for index, row in df.iterrows():
    print("🟢",tc.BOLD_BAKGROUNDs.PURPLE,row['name']+str(row['year_type']),tc.ENDC,"🟢")
    r_roi = ast.literal_eval(row['roi'])
    r_roi = ee.Geometry.Polygon(r_roi, None, False)
    r_date = ast.literal_eval(row['date'])
    r_pp =  row['priority_path']
    r_mc =  float(row['max_cloud'])
    
    r_msn =  float(row['max_snow'])
    r_ms =  int(row['month_span'])
    r_rd =  int(row['retry_days'])

    s2_col,s1_col = s1s2(r_roi , date = r_date , priority_path = r_pp , max_cloud =r_mc , max_snow = r_msn , month_span = r_ms ,retry_days = r_rd , sr = SR,best_orbit=True)
    s1_col = s1_col.map(toLinear) # convers dB values to Linear, and adds band 'VV_lin' to the images of the collection
    print(tc.OKCYAN,"---------------------------S2---------------------------",tc.ENDC)
    sen2_short_print(s2_col)
    print(tc.OKCYAN,"---------------------------S1---------------------------",tc.ENDC)
    sen1_short_print(s1_col)
    
    #=====================DOWNLOAD==========================
    # s1_img = s1_col.mean().clip(r_roi)
    # s1_img = toDb(s1_img) # reads the 'VV_lin' and returns the image with the added band 'VV_db' 
    
    # if s2_col.size().getInfo()< 3:
    #     s2_img = s2_col.sort('roi_cloud_cover').first().clip(r_roi).multiply(0.0001)
    # else:    
    #     s2_img = s2_col.median().clip(r_roi).multiply(0.0001)

    # s1_img_vv = s1_img.select('VV_db')
    # s2_img_12b = s2_img.select(bands)
    
    # S1_FOLDER = S1_FOLDER_TRAIN if row['train_test'] == 'train' else S1_FOLDER_TEST
    # S2_FOLDER = S2_FOLDER_TRAIN if row['train_test'] == 'train' else S2_FOLDER_TEST
        
    # r_name = str(index+1).zfill(3) + '_' + row['name'] + str(row['year_type'])
    # geemap.download_ee_image(s1_img_vv, S1_FOLDER +r_name+".tif",crs='EPSG:4326', scale=10,region = r_roi)
    # geemap.download_ee_image(s2_img_12b, S2_FOLDER +r_name+".tif",crs='EPSG:4326', scale=10,region = r_roi)

🟢  abudhabi21  🟢
 ◍◍Finding S2 
 ◍Single scene coverege was found! 
 ◍◍Finding S1 
mean s2 date:  2021-06-26
final date range:   2021-05-26 2021-07-26 | diff_days -> 61 
◍checking for DESCENDING single scene
Mosiac Covers 96.63861773206382 percent of the roi.
Mosaic Coverege Accepted
Mosaic Coverege Accepted
 Low number of images (col_size=5), we will retry:  
  RETRY: Expanding date range by 15 days
 ◍◍Finding S1 
mean s2 date:  2021-06-26
final date range:   2021-05-18 2021-08-03 | diff_days -> 77 
◍checking for DESCENDING single scene
Mosiac Covers 96.63861773206382 percent of the roi.
Mosaic Coverege Accepted
Mosaic Coverege Accepted
 Low number of images (col_size=6), we will retry:  
  RETRY: Expanding date range by 15 days
 ◍◍Finding S1 
mean s2 date:  2021-06-26
final date range:   2021-05-10 2021-08-11 | diff_days -> 93 
◍checking for DESCENDING single scene
Mosiac Covers 96.63861773206382 percent of the roi.
Mosaic Coverege Accepted
Mosaic Coverege Accepted
 ◍Collection Found

In [ ]:
if IN_COLAB:
    ! du -sh /content/s1_imgs/
    ! du -sh /content/s2_imgs/

4.0K	/content/s1_imgs/
4.0K	/content/s2_imgs/


## Saving downloaded dataset for later use in drive

In [ ]:
# from distutils.dir_util import copy_tree
# copy_tree("/content/s1_imgs", "/content/drive/MyDrive/Colab Notebooks/rgb2sar/dataset/s1")
# copy_tree("/content/s2_imgs", "/content/drive/MyDrive/Colab Notebooks/rgb2sar/dataset/s2")